In [1]:
from utils.split_embeds import *

In [2]:
#base_model_name = "./Data/Bert/fine-tuned-bert-2022-02-21-16-20-51"
base_model_name = "all-mpnet-base-v2"           # #"all-mpnet-base-v2"          #"paraphrase-multilingual-mpnet-base-v2" all-mpnet-base-v2
IsBaseline = True
data_import_path = "./Data/Preprocessed/data_with_filled_explanations_17.2.2022.pickle"
source_col_name = "explanation_prep"  #"explanation_prep"  "statement_explanation_prep"                     #"source_text_shorter"  # "source_text_shorter" source_text
targer_col_name = "shortExplanation_prep"                         #"target_text"
neighbours = 4
data_with_similarity_dir = "./Data/Similarity/data_{}_{}.pickle".format(source_col_name,neighbours)
data_embeddings_dir = "./Data/Embeddings/embed_{}_{}.pkl".format(source_col_name, targer_col_name)
min_no_sentence_source_text = 3 

In [26]:
torch.cuda.get_device_name("cuda:0")

'A100-SXM4-40GB'

In [28]:
data = pd.read_pickle(data_import_path)
len(data)

12891

In [29]:
data.describe()

,id,statementTokensLength,explanationTokensLength,shortExplanationTokensLength,statementexplanationTokensLength
count,12891.000000,12891.000000,12891.000000,12891.000000,12891.000000
mean,6491.162672,18.412691,775.487937,85.352416,793.333721
std,3752.628466,8.066301,288.562379,42.275294,288.266961
min,1.000000,3.000000,23.000000,2.000000,45.000000
25%,3241.500000,13.000000,570.000000,58.000000,588.000000
50%,6481.000000,17.000000,736.000000,80.000000,755.000000
75%,9744.500000,23.000000,934.000000,106.000000,952.000000
max,12994.000000,73.000000,2912.000000,1121.000000,2899.000000


In [30]:
data = split_data(data, source_col_name, targer_col_name)

In [31]:
data_shorter = select_longer_claims(data)
len(data_shorter)

465236

In [32]:
data_shorter.to_pickle("Data/backup/data150.pickle")

In [33]:
data_shorter = pd.read_pickle("Data/backup/data150.pickle")

In [34]:
data_shorter.columns

Index(['id', 'reviewer', 'date', 'statement', 'explanation',
       'shortExplanation', 'truth_o_meter', 'tags', 'url', 'statement_prep',
       'explanation_prep', 'shortExplanation_prep',
       'statement_explanation_prep', 'statementTokensLength',
       'explanationTokensLength', 'shortExplanationTokensLength',
       'statementexplanationTokensLength', 'label', 'source_text',
       'target_text', 'source_text_sentences', 'source_text_sentences_len',
       'source_text_sentences_index'],
      dtype='object')

In [35]:
#model = SentenceTransformer(base_model_name, device = 'cuda:0')
sentences_lst = data_shorter["source_text_sentences"].tolist()

In [36]:
len(data_shorter)

465236

In [37]:
embeddings = embeddings_sentence_bert(sentences_lst, IsBaseline, base_model_name)               #  model.encode(sentences_lst, show_progress_bar=True, batch_size = 500)  

Batches:   0%|          | 0/931 [00:00<?, ?it/s]

Time for creating 465236 embedding vectors 2.740990654627482
Model used :all-mpnet-base-v2


In [38]:
#with open(data_embeddings_dir, 'wb') as f:
#    pickle.dump(embeddings, f)

## Read data and convert huge numpy array to list of lists

In [39]:
#data_shorter = pd.read_pickle("data77.pickle")
#with open(data_embeddings_dir, 'rb') as f:
#    embeddings = pickle.load(f)

In [40]:
data = pd.DataFrame([[i] for i in tqdm(embeddings)]).rename(columns={0:'source_text_sentences_embed_base'})

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 465236/465236 [00:00<00:00, 792851.69it/s]


In [41]:
data_shorter = pd.concat([data_shorter.reset_index(), data.reset_index()], axis=1)

In [42]:
#data_shorter

In [43]:
lof = LocalOutlierFactor(n_neighbors = neighbours, metric = 'cosine')

In [44]:
from tqdm import tqdm
tqdm.pandas()

In [45]:
data_shorter["LOF_base"] = data_shorter.groupby('id',sort = False)['source_text_sentences_embed_base'].progress_apply(get_lof_score).explode().to_list()
data_shorter.to_pickle(data_with_similarity_dir)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12891/12891 [00:11<00:00, 1144.48it/s]


In [46]:
data_shorter.columns

Index(['index', 'id', 'reviewer', 'date', 'statement', 'explanation',
       'shortExplanation', 'truth_o_meter', 'tags', 'url', 'statement_prep',
       'explanation_prep', 'shortExplanation_prep',
       'statement_explanation_prep', 'statementTokensLength',
       'explanationTokensLength', 'shortExplanationTokensLength',
       'statementexplanationTokensLength', 'label', 'source_text',
       'target_text', 'source_text_sentences', 'source_text_sentences_len',
       'source_text_sentences_index', 'index',
       'source_text_sentences_embed_base', 'LOF_base'],
      dtype='object')

# In case we have big data 

## Concatenate without embeddings (if too big numpy array)

In [2]:
data  = pd.read_pickle("./Data/Similarity/data_whole_title_embed_text_title_30K.pickle")
data_shorter  = pd.read_pickle("./Data/Similarity/data_whole_title_embed_text_title_150K.pickle")

In [3]:
data = data[data.type != "train"]

In [4]:
data.drop('source_text_sentences_embed_base', inplace=True, axis=1)
data_shorter.drop('source_text_sentences_embed_base', inplace=True, axis=1)

In [6]:
data_res = pd.concat([data, data_shorter], axis=0)

In [10]:
data_res.to_pickle(data_with_similarity_dir)